In [1]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer.noise import NoiseModel
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_aer import Aer, AerSimulator
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit_algorithms.utils import algorithm_globals
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from QArithmetic import add
from qiskit_aer.noise import NoiseModel

service = QiskitRuntimeService(channel="ibm_quantum", token="1336d138b570f37346d77ce4ace61b5f0018533ac50ded70e1d593d493669775f85bc98e77ca6c23bdf2bbbe590d93dda6e67d7ab5b8f23cfb1b78270ffaf562")
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=10)
 
sampler = Sampler(mode=backend)

a = QuantumRegister(2)
b = QuantumRegister(3)
cb = ClassicalRegister(3)
qc = QuantumCircuit(a, b, cb)

# Numbers to add.
# qc.x(a[1]) # a = 01
qc.x(a[0])
# qc.x(a[3])
qc.h(b[0]) # b = 011 / 010
qc.x(b[1])
# qc.x(b[3])

# Add the numbers, so |a>|b> to |a>|a+b>.
add(qc, a, b, 2)

# Measure the results.


In [2]:
from qiskit.quantum_info import hellinger_fidelity
from qiskit_aer.primitives import SamplerV2 as Sampler

measured_circuit = qc.copy()
measured_circuit.measure_all()

noise_model = NoiseModel()
cx_depolarizing_prob = 0.02
noise_model.add_all_qubit_quantum_error(
    depolarizing_error(cx_depolarizing_prob, 2), ["cx"]
)

noisy_sampler = Sampler(options=dict(backend_options=dict(noise_model=noise_model)))
# The circuit needs to be transpiled to the AerSimulator target
pass_manager = generate_preset_pass_manager(3, AerSimulator())
isa_circuit = pass_manager.run(measured_circuit)
pub = (isa_circuit)
job = noisy_sampler.run([pub])
result = job.result()
pub_result = result[0]
pub_result.data.meas.get_counts()

# exp: 100 01, 011 01

NameError: name 'depolarizing_error' is not defined

In [27]:
print(qc)

       ┌───┐                                                              »
q20_0: ┤ X ├───────────────────────────────────────────────■──────────────»
       └───┘                                               │              »
q20_1: ──────────────────────────────────────■─────────────┼────────■─────»
       ┌───┐                                 │       ┌───┐ │        │     »
q21_0: ┤ H ├──────────■─────────────■────────┼───────┤ H ├─┼────────┼─────»
       ├───┤          │       ┌───┐ │P(π/2)  │       └───┘ │        │P(π) »
q21_1: ┤ X ├─■────────┼───────┤ H ├─■────────┼─────────────┼────────■─────»
       ├───┤ │P(π/2)  │P(π/4) └───┘          │P(π/2)       │P(π/4)        »
q21_2: ┤ H ├─■────────■──────────────────────■─────────────■──────────────»
       └───┘                                                              »
 c3: 3/═══════════════════════════════════════════════════════════════════»
                                                                          »
«           